## Наївний Баєс

In [1]:
# %pip install torch pandas transformers scikit-learn matplotlib nltk pymorphy2 pymorphy2-dicts-uk

In [2]:
# import nltk
# nltk.download("punkt")

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import numpy as np
import pandas as pd

from preprocessing import (get_x, 
                           get_y, 
                           get_vocab_size, 
                           tokenize_x, 
                           tokenize_titles,
                           balance_data, 
                           create_dictionary, 
                           lemmatize_word, 
                           is_not_punctuation, 
                           cat_titles_and_texts,
                           get_x1)

In [4]:
df = pd.read_csv("translated.csv")
x = get_x(df)
y = get_y(df)
x_titles = get_x1(df)

In [5]:
df_zepopo = balance_data(pd.read_csv("data_set_4.csv"))
# df_zepopo = pd.read_csv("data_set_4.csv")
# df_zepopo = df_zepopo.sample(frac=1).iloc[::4]
print(len(df_zepopo))
x1 = get_x(df_zepopo)
y1 = get_y(df_zepopo)
x1_titles = get_x1(df_zepopo)

Int64Index([ 706, 5325, 1665, 6675, 5605, 2317, 1691, 5030,  318, 3745,
            ...
            6500,  422, 1220, 3439, 3877, 7724, 6732,  102, 2418, 7796],
           dtype='int64', length=2498)
4913


In [6]:
df_realdata = pd.read_csv("new_real_news.csv")
print(len(df_realdata))
x2 = get_x(df_realdata)
y2 = get_y(df_realdata)
x2_titles = get_x1(df_realdata)

1188


In [7]:
# merging two datasets together
x += x1
x += x2

y += y1
y += y2

x_titles += x1_titles
x_titles += x2_titles

x = cat_titles_and_texts(texts=x, titles=x_titles)

In [8]:
X_train, X_test, y_train, y_test, X_titles_train, X_titles_test = train_test_split(
    x, y, x_titles, test_size=0.2, random_state=42
)

In [9]:
import json
# loading an existing dictionary
with open("dictionary.json", "r", encoding="utf-8") as f:
    dictionary = json.load(f)

In [10]:
X_train, X_test = np.array(tokenize_x(X_train, dictionary=dictionary), dtype=float), np.array(tokenize_x(X_test, dictionary=dictionary), dtype=float)
X_titles_train, X_titles_test = np.array(tokenize_x(X_titles_train, dictionary=dictionary), dtype=float), np.array(tokenize_x(X_titles_test, dictionary=dictionary), dtype=float)  

In [11]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [12]:
print(y[:5])

[1.0, 1.0, 1.0, 1.0, 1.0]


In [13]:
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)

### Оцінка моделі

*Метрики*:
- Accuracy
- Precision
- Recall 
- f1

In [14]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score)

In [15]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [16]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1:", f1)

Accuracy: 0.6153549382716049
Precision: 0.6731107205623902
Recall: 0.32077051926298156
f1: 0.43448667044809985


### Тестова вибірка

In [17]:
import json
with open("test_set.json", "r", encoding="utf-8") as f: 
    test_texts = json.load(f)

fake_texts = [t["text"] for t in test_texts if t["label"] == "Fake"]
fake_titles = [t["title"] for t in test_texts if t["label"] == "Fake"]
real_texts = [t["text"] for t in test_texts if t["label"] == "Real"]
real_titles = [t["text"] for t in test_texts if t["label"] == "Real"]

In [18]:
df = pd.DataFrame(columns = ["ukr_text", "label", "title_ukr"])
for i, text in enumerate(fake_texts): 
    df.loc[i] = {
        "ukr_text": text, 
        "label": "Fake", 
        "title_ukr": fake_titles[i]
    }

for i, text in enumerate(real_texts): 
    df.loc[i+20] = {
        "ukr_text": text, 
        "label": "Real", 
        "title_ukr": real_titles[i]
    }

In [19]:
test_data_x = np.array(tokenize_x(get_x(df), dictionary=dictionary), dtype="float")
test_data_x_titles = np.array(tokenize_x(get_x1(df), dictionary=dictionary), dtype="float")

test_data_x = test_data_x + test_data_x_titles
test_data_y = np.array(get_y(df))

In [20]:
test_pred = gnb.predict(test_data_x)

In [21]:
accuracy = accuracy_score(test_data_y, test_pred)
precision = precision_score(test_data_y, test_pred)
recall = recall_score(test_data_y, test_pred)
f1 = f1_score(test_data_y, test_pred)

In [22]:
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1:", f1)

Accuracy: 0.375
Precision: 0.25806451612903225
Recall: 0.4
f1: 0.3137254901960784
